World Bank Data Analysis 

In this project, I cleaned up the data from the World Bank. The dataset was initially in JSON format and can be found on this website: http://jsonstudio.com/resources/. 



I conducted three tasks with the dataset: 

1) I found the 10 countries with most projects.

2) I found the top 10 major project themes, using column 'mjtheme_namecode'.

3) In 2, I noticed that some entries have only the code and the name is missing, so I created a new dataframe with the missing names filled in.

In [1]:
## I will start by importing the dataset and the pandas and JSON packages. 

In [2]:
# Imports
import pandas as pd
import json
from pandas.io.json import json_normalize

In [3]:
# This reads the creates a Pandas Dataframe out of the json file. 
df = pd.read_json('world_bank_projects.json')
df.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


Task 1: 

Find the 10 countries with the most projects

In [5]:
# The following outputs the countries with the most projects. 

df.countryshortname.value_counts().head(10)

# There is one problem, though: it includes Africa as a country
# in 9th spot. 
# Several projects which are in multiple African countries have 
# "Africa" stored as their country. 

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64

In [6]:
# The database does not store which specific
# African countries are involved in these projects with Africa as the label,
# so I do not have access to which specific African countries these projects are in, 
# meaning I can do little with those data values. 
# Thus I will create a new data frame that excludes the projects
# with Africa as their country label and find its 10 most common countries. 

# df_notAfr is the copy of the dataframe excluding projects labelled with 
# Africa as their company name. 
df_notAfr = df[df.countryshortname != "Africa"]
df_notAfr.countryshortname.value_counts().head(10)

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Pakistan               9
Name: countryshortname, dtype: int64

Task 2: 

Find the 10 major project themes, using column 'mjtheme_namecode'

In [7]:
# A detailed inspection of mjtheme_namecode in df reveals that
# each project contains list of dictionaries containing the code
# values and corresponding code name (with some names missing, 
# which I will address in Task 3).

# Note: Upon closer inspection, all the codes are between 
# 1 and 11, and all filled code names are consistent. 
# For example, all code 8 values with a non-empty name are
# named consistently. 
# Also note: Some projects include code repeats. For example, the third
# project includes Code 7 twice. The counting method below will make 
# that multiple projects from one code are only counted once. 

# This is the highest code value. 
code_max = 11

df.mjtheme_namecode.head(20)

0     [{'code': '8', 'name': 'Human development'}, {...
1     [{'code': '1', 'name': 'Economic management'},...
2     [{'code': '5', 'name': 'Trade and integration'...
3     [{'code': '7', 'name': 'Social dev/gender/incl...
4     [{'code': '5', 'name': 'Trade and integration'...
5     [{'code': '6', 'name': 'Social protection and ...
6     [{'code': '2', 'name': 'Public sector governan...
7     [{'code': '11', 'name': 'Environment and natur...
8     [{'code': '10', 'name': 'Rural development'}, ...
9     [{'code': '2', 'name': 'Public sector governan...
10    [{'code': '10', 'name': 'Rural development'}, ...
11    [{'code': '10', 'name': 'Rural development'}, ...
12                          [{'code': '4', 'name': ''}]
13    [{'code': '5', 'name': 'Trade and integration'...
14    [{'code': '6', 'name': 'Social protection and ...
15    [{'code': '10', 'name': 'Rural development'}, ...
16    [{'code': '10', 'name': 'Rural development'}, ...
17    [{'code': '8', 'name': 'Human development'

In [8]:
# Counts the number of projects that invoke each code, 
# storing that as a dictionary and then converting it to a Series. 

# md, short of 'master dictionary', will store the number of
# projects that invoke each code number. 
md = { }

# Initialize each code's count at zero. 
for i in range(1, code_max+1):
    md[i] = 0
    
# Move through each set of stored codes for each project to 
# count the number of projects that include each code. 
for x in range(len(df)):
    # d stores the codes used for each project
    d = {}
    for y in range(len(df.mjtheme_namecode[x])):
        d[int(df.mjtheme_namecode[x][y]['code'])] = 1
    
    # Adds the d to overall count in md
    for i in d.keys():
        md[i] += 1

# mdf, short for 'master dataframe', stores the number of projects
# that invoke each code number, converting md to a series 
# that I can now use Pandas to analyze the top occurences. 
mdf = pd.Series(md)
mdf

1      33
2     140
3      14
4     119
5      61
6     116
7     114
8     128
9      40
10    148
11    157
dtype: int64

In [10]:
# The 10 most frequently invoked codes can now be easily obtained 
# by simply selecting the top 10 counts in mdf. 

mdf.sort_values(ascending = False).head(10)

11    157
10    148
2     140
8     128
4     119
6     116
7     114
5      61
9      40
1      33
dtype: int64

Task 3: 

Created a new dataframe with the missing names filled in.

In [15]:
# Printed are all the codes for all the projects. Looking through this 
# large output, one notices that several code names are blank. 
# I will fill in the gaps. 

# limit will provide how many times it will print through the long output, and counter will increase until it hits limit.
limit = 25
counter = 0

for x in range(len(df.mjtheme_namecode)):
    for y in range(len(df.mjtheme_namecode[x])):
        print(df.mjtheme_namecode[x][y])
        counter += 1
    if(counter >= limit):
        break


{'code': '8', 'name': 'Human development'}
{'code': '11', 'name': ''}
{'code': '1', 'name': 'Economic management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '5', 'name': 'Trade and integration'}
{'code': '2', 'name': 'Public sector governance'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': '5', 'name': 'Trade and integration'}
{'code': '4', 'name': 'Financial and private sector development'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '6', 'name': ''}
{'code': '2', 'name': 'Public sector governance'}
{'code': '4', 'name': 'Financial and private sector development'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '8', 'name': ''}
{'code': '10', 'name': 'Rural development'}
{'code': '7', 'name': ''}


In [17]:
# code_names are the list of the proper code names in order. 
# Note: The first list value is empty because there is no name for code: '0'. 
# Thus the first non-empty code name corresponds with code_names[1]. 

code_names = [ "",
    'Economic management', 
    'Public sector governance', 
    'Rule of law',
    'Financial and private sector development', 
    'Trade and integration', 
    'Social protection and risk management', 
    'Social dev/gender/inclusion', 
    'Human development', 
    'Urban development', 
    'Rural development', 
    'Environment and natural resources management'
]

In [18]:
# df_copy will be the copy of the dataframe with the correct names
df_copy = df.copy()

# Looks through each code name, and if it is empty, stores the proper name in its place
for x in range(len(df_copy.mjtheme_namecode)):
    for y in range(len(df_copy.mjtheme_namecode[x])):
        if(df_copy.mjtheme_namecode[x][y]['name'] == ""):
            df_copy.mjtheme_namecode[x][y]['name'] = code_names[int(df_copy.mjtheme_namecode[x][y]['code'])]

In [19]:
# Inspecting the code names of df_copy, one see that none are empty.

# limit will provide how many times it will print through the long output, and counter will increase until it hits limit.
limit = 25
counter = 0

for x in range(len(df_copy.mjtheme_namecode)):
    for y in range(len(df_copy.mjtheme_namecode[x])):
        print(df_copy.mjtheme_namecode[x][y])
        counter += 1
    if(counter >= limit):
        break

{'code': '8', 'name': 'Human development'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '1', 'name': 'Economic management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '5', 'name': 'Trade and integration'}
{'code': '2', 'name': 'Public sector governance'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': '5', 'name': 'Trade and integration'}
{'code': '4', 'name': 'Financial and private sector development'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '2', 'name': 'Public sector governance'}
{'code': '4', 'name': 'Financial and private sector development'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '8', '